# getCPI
---
### getCPI(endDate)
* endDate : (AD)YYYY-MM or (民國)YYY-MM (ex.2000-01 or 089-01)

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import re

In [2]:
url = 'https://www.stat.gov.tw/lp.asp?CtNode=489&CtUnit=1818&BaseDSD=29&nowPage=1&pagesize=25'
html = requests.get(url)
print(html.text)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="Zh-tw" ><html xmlns:msxsl="urn:schemas-microsoft-com:xslt" xmlns:user="urn:user-namespace-here">
<head>
<META http-equiv="Content-Type" content="text/html; charset=UTF-8">

<meta http-equiv="X-UA-Compatible" content="IE=edge">
<title>中華民國統計資訊網（專業人士）</title>
<META NAME="keywords" CONTENT="Price Index,National Income,Unemployment Rate,Census,Social Indicator,Economic Growth Rate,Family Income and Expenditure,Standard Classification,Labor Froce,Input-Output">
<META NAME="description" CONTENT="Statistical Information Network of the Republic of China This website serves as a national statistical portal of the Republic of China. The website is responsible for collecting, compiling and disseminating the nation's statistical information and presenting it in the form of the latest economic and social indicators, updated reports, analysis and statistical publicat

In [3]:
soup = bs(html.text, 'html.parser')
print(soup.prettify)

<bound method Tag.prettify of 
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="Zh-tw"><html xmlns:msxsl="urn:schemas-microsoft-com:xslt" xmlns:user="urn:user-namespace-here">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>中華民國統計資訊網（專業人士）</title>
<meta content="Price Index,National Income,Unemployment Rate,Census,Social Indicator,Economic Growth Rate,Family Income and Expenditure,Standard Classification,Labor Froce,Input-Output" name="keywords"/>
<meta content="Statistical Information Network of the Republic of China This website serves as a national statistical portal of the Republic of China. The website is responsible for collecting, compiling and disseminating the nation's statistical information and presenting it in the form of the latest economic and social indicators, updated reports, analysis and statist

In [4]:
text = soup.find_all('a', title=re.compile("CPI|WPI"))
text

[<a href="ct.asp?xItem=43311&amp;ctNode=489&amp;mp=4" title="107年8月消費者物價指數(CPI)年增率漲1.53％，躉售物價指數(WPI)年增率漲6.83％">107年8月消費者物價指數(CPI)年增率漲1.53％，躉售物價指數(WPI)年增率漲6.83％</a>,
 <a href="ct.asp?xItem=43116&amp;ctNode=489&amp;mp=4" title="107年7月消費者物價指數(CPI)年增率漲1.75％，躉售物價指數(WPI)年增率漲7.02％">107年7月消費者物價指數(CPI)年增率漲1.75％，躉售物價指數(WPI)年增率漲7.02％</a>,
 <a href="ct.asp?xItem=42997&amp;ctNode=489&amp;mp=4" title="107年6月消費者物價指數(CPI)年增率漲1.31％，躉售物價指數(WPI)年增率漲6.65％">107年6月消費者物價指數(CPI)年增率漲1.31％，躉售物價指數(WPI)年增率漲6.65％</a>,
 <a href="ct.asp?xItem=42884&amp;ctNode=489&amp;mp=4" title="107年5月消費者物價指數(CPI)年增率漲1.64％，躉售物價指數(WPI)年增率漲5.61％">107年5月消費者物價指數(CPI)年增率漲1.64％，躉售物價指數(WPI)年增率漲5.61％</a>,
 <a href="ct.asp?xItem=42774&amp;ctNode=489&amp;mp=4" title="107年4月消費者物價指數(CPI)年增率漲1.98％，躉售物價指數(WPI)年增率漲2.53％">107年4月消費者物價指數(CPI)年增率漲1.98％，躉售物價指數(WPI)年增率漲2.53％</a>,
 <a href="ct.asp?xItem=42655&amp;ctNode=489&amp;mp=4" title="107年3月消費者物價指數(CPI)年增率漲1.57％，躉售物價指數(WPI)年增率漲0.46％">107年3月消費者物價指數(CPI)年增率漲1.57％，躉售物價指數(WPI)年增率漲0.46％</a>,
 <a href="

In [5]:
text[0].string

'107年8月消費者物價指數(CPI)年增率漲1.53％，躉售物價指數(WPI)年增率漲6.83％'

In [6]:
re.split('年|月|增率|，|％', text[0].string)

['107', '8', '消費者物價指數(CPI)', '', '漲1.53', '', '躉售物價指數(WPI)', '', '漲6.83', '']

In [7]:
year = []
month = []
CPI = []
WPI = []
for i in range(len(text)):
    temp = re.split('年|月|增率|，|％', text[i].string)
    year.append(temp[0])
    month.append(temp[1])
    CPI.append(temp[4][1:] if temp[4][0]=='漲' else '-'+temp[4][1:])
    WPI.append(temp[-2][1:] if temp[-2][0]=='漲' else '-'+temp[-2][1:])
info = pd.DataFrame({'Year' : year, 'Month' : month, 'CPI (%)': CPI, 'WPI (%)' : WPI})\
            [['Year', 'Month', 'CPI (%)', 'WPI (%)']]
info

,Year,Month,CPI (%),WPI (%)
0,107,8,1.53,6.83
1,107,7,1.75,7.02
2,107,6,1.31,6.65
3,107,5,1.64,5.61
4,107,4,1.98,2.53
5,107,3,1.57,0.46
6,107,2,2.19,-0.27
7,107,1,0.88,-0.74
8,106,12,1.21,0.24
9,106,11,0.35,1.57


In [8]:
info.to_csv('CPI.csv', index = False)